In [ ]:
# =============================================================================
# Authors: Deepthi Kuttichira and Pubudu Sanjeewani Thihagoda Gamage
# Date: 20-08-2024
# Title: A Novel Non-Iterative Training Method for CNN Classifiers Using Gram-Schmidt Process
# License: MIT License
# =============================================================================

# Import necessary libraries
import numpy as np
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset (only used here to get labels)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Load the precomputed feature matrix (A) from file and remove zero-only columns (Change the file path to the actual location on the system)
A = np.load(r'C:\Users\sanje\Desktop\Griffith\code\pretrained-MNIST\mnisttrain_2000relu2.npy')
A = A[:, np.any(A != 0, axis=0)] 

# Set label vector b using MNIST training labels
b = y_train

# Display shapes of A and b
print(f"Shape of feature matrix A: {A.shape}")
print(f"Shape of label vector b: {b.shape}")

# Gram-Schmidt QR Factorization Function
def gram_schmidt_qr(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    for j in range(n):
        v = A[:, j]
        for i in range(j):
            q = Q[:, i]
            R[i, j] = np.dot(q, v)
            v = v - R[i, j] * q
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]
    return Q, R

# Back-substitution Function to solve upper triangular system Rx = c
def back_substitution(R, c):
    n = len(c)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = c[i]
        for j in range(i+1, n):
            x[i] -= R[i, j] * x[j]
        x[i] /= R[i, i]
    return x

# Get all unique classes in MNIST (0 to 9)
unique_classes = np.unique(b)
print("Unique Classes")
print(unique_classes)

# Compute and store weights for each class using one-vs-all strategy
weights = []

for cls in unique_classes:
    print(cls)
    # Create binary label vector: 1 for current class, 0 otherwise
    b_current_class = np.where(b == cls, 1, 0)
    
    # Perform QR factorization of A
    Q, R = gram_schmidt_qr(A)
    
    # Compute Q^T * b_current_class
    c = np.dot(Q.T, b_current_class)
    
    # Solve Rx = c using back-substitution
    x = back_substitution(R, c)
    
    # Append weight vector for this class
    weights.append(x)

# Final output and saving the weights
print("Weights for each class:", weights)

# Convert list to numpy array and save to .npy file
w=np.array(weights)
np.save("qrweightmnist_2000relu2.npy",w)

Shape of feature matrix A: (60000, 946)
Shape of label vector b: (60000,)
Unique Classes
[0 1 2 3 4 5 6 7 8 9]
0
1
2
3
4
5
6
7
8
9
Weights for each class: [array([ 5.98827843e-01,  2.84474832e-01, -3.86894010e-03, -8.53481953e-02,
        8.10783752e-01,  1.81762032e-01,  4.26984734e-01, -1.81552956e-01,
        9.28395810e-02, -2.79705007e-01,  1.15894718e+00,  8.63883338e-02,
       -6.45647960e-01, -1.03436498e+00, -2.96198264e-02, -6.04649458e-01,
       -9.87148221e-01, -5.54028358e-01, -1.55740302e-01, -3.46909398e-01,
       -3.22387448e-01,  1.39441522e+00,  2.18104586e-01,  1.51116874e-01,
        4.23873056e-01,  4.71536449e-01, -3.84877334e-01, -5.75229242e-02,
        4.73860346e+00,  1.14281612e+00, -8.67286644e-01, -1.09799724e+00,
        2.71054234e+00,  2.56331830e-01,  2.88494241e-01, -1.08304745e+00,
        2.37469937e+00,  7.19034115e-01, -2.19293753e-01, -4.92259547e-01,
        1.41980815e+00,  1.05696732e+00,  2.22343249e-01,  1.16032683e-01,
        6.17427177e